# LLM Classifiers

**Goal:** Given a sentence as input, classify it as either a prediction or non-prediction.

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from classification_models import EvaluationMetric
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Data

In [3]:
print("======= LOAD DATA =======")

======= LOAD DATA =======


In [4]:
base_data_path = os.path.join(notebook_dir, '../data/')
combine_data_path = os.path.join(base_data_path, 'combined_generated_fin_phrase_bank')
X_test_set_path = os.path.join(combine_data_path, 'x_test_set-v1.csv')
y_test_set_path = os.path.join(combine_data_path, 'y_test_set-v1.csv')

In [5]:
X_test_df = DataProcessing.load_from_file(X_test_set_path, 'csv')
X_test_df.drop(columns=['Unnamed: 0'], inplace=True)
X_test_df.head(7)

,Base Sentence,Sentence Label,Embedding,Normalized Embeddings
0,"On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.",1,[-2.67128736e-01 3.99917126e-01 1.51789542e-02 -8.48717690e-02\n -9.08976942e-02 -2.59267748e-03 5.57870697e-03 -1.11984558e-01\n 5.50340861e-02 1.75547993e+00 -2.54974365e-01 -1.80750433e-02\n 2.53512263e-02 -5.75582273e-02 -1.17895454e-01 -3.12662870e-02\n -7.41217807e-02 1.10751092e+00 -1.34572908e-01 -2.01131892e-03\n 5.15997782e-02 1.16336204e-01 -8.87980685e-03 -1.33108839e-01\n 4.78714556e-02 5.70764579e-02 -1.72738910e-01 4.16884683e-02\n 2.07518153e-02 4.11881283e-02 7.36815408e-02 -1.50440494e-02\n 3.74726136e-03 1.82219267e-01 -6.20155334e-02 -4.09787893e-03\n 5.32624172e-03 8.10612142e-02 -4.86277193e-02 -1.43699899e-01\n -7.53155202e-02 -2.91622225e-02 8.86417702e-02 -5.73523762e-03\n 5.72941788e-02 5.58102839e-02 -1.44644111e-01 -6.10220470e-02\n 6....,[-1.92454374e+00 1.91188228e+00 6.52823076e-02 -1.09791911e+00\n -1.71294856e+00 5.92892647e-01 1.92068860e-01 -1.46676123e+00\n 1.23968937e-01 -3.15836072e-02 6.55651987e-01 -5.98727763e-01\n -3.60853940e-01 -9.42990124e-01 -7.83263743e-01 9.00564417e-02\n -1.02269506e+00 3.85037921e-02 -3.46616179e-01 3.87996107e-01\n 4.86750692e-01 8.39973629e-01 -1.23200588e-01 -6.20292664e-01\n 1.05327025e-01 -1.63882062e-01 -7.20112205e-01 2.79392928e-01\n -3.39587964e-02 -5.77544093e-01 1.50556660e+00 -4.31914985e-01\n 1.02688305e-01 1.54282737e+00 -2.37315607e+00 2.74423778e-01\n 4.23447758e-01 4.65204388e-01 -6.98811769e-01 -1.86601353e+00\n -1.40664899e+00 -1.21409535e+00 4.51989844e-02 3.71334434e-01\n 7.22978592e-01 9.42269504e-01 -1.18480968e+00 -2.19736397e-01\n 5....
1,"Operating profit for the three-month period increased from EUR1 .2 m , while revenue increased from EUR20 .2 m , as compared to the corresponding period in 2005 .",0,[-8.39660242e-02 1.80937588e-01 -4.57216762e-02 1.31758088e-02\n -1.15853874e-02 -5.86525165e-02 -8.70665710e-04 3.26534510e-02\n 1.16215296e-01 1.84254622e+00 -2.33168826e-01 1.67165652e-01\n -6.99797019e-05 -2.04802342e-02 4.41387780e-02 -8.78530294e-02\n 3.87666002e-02 1.38155854e+00 -1.93677381e-01 2.67565437e-02\n 8.30405392e-03 2.85588354e-02 -9.71042439e-02 -2.32131258e-02\n 4.73141745e-02 -5.10274470e-02 -9.36960801e-02 4.17356864e-02\n -1.44721614e-02 1.21533826e-01 -6.50100559e-02 -2.13543437e-02\n -8.56587365e-02 2.31346134e-02 4.07231599e-02 -7.77563825e-02\n -1.08038373e-01 -2.93932352e-02 2.08473653e-02 7.65744643e-03\n -1.03914235e-02 4.45170105e-02 7.58761391e-02 -4.45002131e-03\n -9.02475566e-02 1.67677191e-03 -5.21407127e-02 7.16749728e-02\n -2....,[ 0.32650337 -1.0380536 -0.91864634 0.51113987 -0.65212625 -0.29796603\n 0.07093044 0.41100782 0.9220639 0.2662444 0.95830584 1.7495745\n -0.7324779 -0.3097109 0.88611746 -0.9876781 1.0547875 1.9621172\n -1.3324827 0.875457 -0.40856755 -0.4626243 -1.5807649 0.94958717\n 0.0967818 -1.6057067 0.50081563 0.2801942 -0.6300226 0.58435524\n -1.0352651 -0.5324218 -1.302615 -0.7046011 -0.29519314 -0.9691\n -1.0828011 -1.2725768 0.4982816 0.4064646 -0.23578587 -0.05474434\n -0.12900919 0.39317027 -2.0725603 0.0272575 0.11121429 1.404259\n -0.9829128 0.3590206 0.32491672 -1.1381576 -0.5637724 0.83665544\n 0.79479074 -0.45303088 -0.28690377 -1.7320254 -1.0912753 1.3369948\n 0.07836758 -0.26163587 -0.870195 0.77505666 -1.885778 -0.8...
2,"Cybersecurity threats should stay same in 2026-06-01, according to the National Security Agency director, Dr. Sophia Rodriguez.",1,[-9.97022167e-02 2.27282479e-01 8.88563171e-02 1.93849038e-02\n 8.27627033e-02 -6.11012615e-03 6.32572249e-02 -1.25566736e-01\n -9.99972895e-02 1.72420835e+00 -3.31175238e-01 -4.78917100e-02\n 4.07287441e-02 -8.05337653e-02 -1.63093254e-01 -3.94880436e-02\n -1.27118714e-02 8.93040419e-01 -5.53227402e-02 7.60172233e-02\n 1.45164803e-01 1.45717368e-01 6.82817474e-02 -7.75945783e-02\n 5.19648008e-02 2.36863736e-02 -1.45081267e-01 -3.98

In [6]:
y_test_df = DataProcessing.load_from_file(y_test_set_path, 'csv')
y_test_df.drop(columns=['Unnamed: 0'], inplace=True)
# print(f"\t{y_test_df.head(7)}")

In [7]:
print(f"\tShape: {X_test_df.shape}, \nSubset of Data:{X_test_df.head(3)}")
# df.shape, df.head(3)

print(f"\tShape: {y_test_df.shape}, \nSubset of Data:{y_test_df.head(3)}")
# df.shape, df.head(3)

	Shape: (21, 4), 
Subset of Data:                                                                                                                                                        Base Sentence  \
0                                    On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.   
1  Operating profit for the three-month period increased from EUR1 .2 m , while revenue increased from EUR20 .2 m , as compared to the corresponding period in 2005 .   
2                                     Cybersecurity threats should stay same in 2026-06-01, according to the National Security Agency director, Dr. Sophia Rodriguez.   

   Sentence Label  \
0               1   
1               0   
2               1   

                                                                                                                                                                                                             

In [8]:
# for idx, row in X_test_df.iterrows():
#     text = row['Base Sentence']
#     label = row['Sentence Label']
#     embedding = row['Embedding']
#     print(type(embedding))
#     norm_embedding = row['Normalized Embeddings']
#     if idx < 7:
#         print(f"{idx}\n Sentence: {text}\n Label: {label}\n Embeddings Shape: {embedding}\n\t Embeddings Subset [:6]: {embedding[:6]} \n Norm Embeddings: {norm_embedding.shape}, \n\tNorm Embeddings Subset [:6]: {norm_embedding[:6]}")
# print(f"\tShape: {y_test_df.shape}, \nSubset of Data:{y_test_df.head(7)}")

## Load Prompt

In [9]:
# prediction_properties = PredictionProperties.get_prediction_properties()
# prediction_requirements = PredictionProperties.get_requirements()
# system_identity_prompt = "You are an expert at identifying specific types of sentences by knowing the sentence format."
# prediction_examples_prompt = """Some examples of predictions in the PhraseBank dataset are
#     1. According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
#     2. According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
#     3. Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
# """
# non_prediction_examples_prompt = """Some examples of non-predictions in the PPhraseBank dataset are
#     1. Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
#     2. Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .
#     3. STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
# """
# # goal_prompt = "Given the above, identify the prediction."

# base_prompt = f"""{system_identity_prompt} The sentence format is based on: 
    
#     {prediction_properties}
#     Enforce: {prediction_requirements}
#     Know: {prediction_examples_prompt}
#     Know: {non_prediction_examples_prompt}

# """
# base_prompt

In [10]:
prompt_1 = """ 

Role: 
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow." (Yes)
- "The stock market is expected to rise next quarter." (Yes)
- "I am going to the store." (No)
- “Lakers will win the championship. ”(Yes)

A prediction may contain: source, target, date, outcome.
"""

## Models

In [11]:
tgmf = TextGenerationModelFactory()

# Option 1: Specific models
models = tgmf.create_instances(['llama-3.1-8b-instant', 'llama-3.3-70b-versatile', 'llama-3.3-70b-instruct'])

# Option 2: All Groq models
# models = tgmf.create_instances(tgmf.get_groq_model_names())

# Option 3: All NaviGator models
# models = tgmf.create_instances(tgmf.get_navigator_model_names())

# Option 4: All available models
# models = tgmf.create_instances()

# Option 5: Mix and match
# custom_models = ['llama-3.1-70b-instruct', 'mistral-small-3.1', 'llama-3.1-8b-instant']
# models = tgmf.create_instances(custom_models)
models

In [12]:
import json
import re

def parse_json_response(response):
    """Parse JSON response from LLM to extract label and reasoning"""
    try:
        # Extract JSON if there's extra text
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            return data.get('label'), data.get('reasoning')
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None, None

In [13]:
def llm_certifier(data: str, base_prompt: str, model):
    
        prompt = f""" Given this: {base_prompt}. Also given the sentence '{data}', your task is to analyze the sentence and determine if it is a prediction. If prediction, generate label as 1 and if non-prediction generate label as 0.
        Respond ONLY with valid JSON in this exact format:
        {{"label": 0, "reasoning": "your explanation here"}}
        Examples:
        - "It will rain tomorrow." → {{"label": 1, "reasoning": "Contains the future tense words 'will' and 'tomorrow'"}}
        - "The stock market is expected to rise next quarter." → {{"label": 1, "reasoning": "Contains future tense words 'is expected'"}}
        - "I am going to the store." → {{"label": 0, "reasoning": "Does not contain a future tense word"}}
        - "Lakers will win the championship." → {{"label": 1, "reasoning": "Contains the future tense word 'will'"}}
        """
        idx = 1
        if idx == 1:
            #   print(f"\tPrompt: {prompt}")
              idx = idx + 1
        input_prompt = model.user(prompt)
        raw_text_llm_generation = model.chat_completion([input_prompt])
        
        # Parse the JSON response
        label, reasoning = parse_json_response(raw_text_llm_generation)
        
        return raw_text_llm_generation, label, reasoning

In [14]:
print("======= PROMPT + MODEL -> LABEL and REASONING =======")

======= PROMPT + MODEL -> LABEL and REASONING =======


In [15]:
    # content : meta :: text : meta_data
results = []
for idx, row in X_test_df.iterrows():
    text = row['Base Sentence']
    print(f"{idx} --- Sentence: {text}")
    for model in models:
        # print(model.__name__())
        raw_response, llm_label, llm_reasoning = llm_certifier(text, prompt_1, model)
        print(f"\tModel: {model.__name__()}\n\t\tLabel:', {llm_label}\n\t\tReasoning:', {llm_reasoning}")
        result = (text, raw_response, llm_label, llm_reasoning, model.__name__())
        results.append(result)


0 --- Sentence: On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.
	Model: llama-3.1-8b-instant
		Label:', 1
		Reasoning:', Contains future tense words 'will likely increase' and specifies a date 'August 15, 2024', a target 'customer satisfaction ratings at Amazon', and a source 'marketing expert David Lee'
	Model: llama-3.3-70b-versatile
		Label:', 1
		Reasoning:', Contains the future tense words 'will likely increase' and a specific date 'August 15, 2024', indicating a prediction about the future
	Model: llama-3.3-70b-instruct
		Label:', 1
		Reasoning:', Contains the future tense words 'will likely increase' and a specific future date 'August 15, 2024', indicating a prediction about the future.
1 --- Sentence: Operating profit for the three-month period increased from EUR1 .2 m , while revenue increased from EUR20 .2 m , as compared to the corresponding period in 2005 .
	Model: llama-3.1-8b-instant
		Label:

In [16]:
results

[('On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.',
  '{"label": 1, "reasoning": "Contains future tense words \'will likely increase\' and specifies a date \'August 15, 2024\', a target \'customer satisfaction ratings at Amazon\', and a source \'marketing expert David Lee\'"}',
  1,
  "Contains future tense words 'will likely increase' and specifies a date 'August 15, 2024', a target 'customer satisfaction ratings at Amazon', and a source 'marketing expert David Lee'",
  'llama-3.1-8b-instant'),
 ('On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.',
  '{"label": 1, "reasoning": "Contains the future tense words \'will likely increase\' and a specific date \'August 15, 2024\', indicating a prediction about the future"}',
  1,
  "Contains the future tense words 'will likely increase' and a specific date 'August 15, 2024', indicati

In [17]:
# groupby text

In [18]:
results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'raw_response', 'llm_label', 'llm_reasoning', 'llm_name'])
results_with_llm_label_df

,text,raw_response,llm_label,llm_reasoning,llm_name
0,"On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.","{""label"": 1, ""reasoning"": ""Contains future tense words 'will likely increase' and specifies a date 'August 15, 2024', a target 'customer satisfaction ratings at Amazon', and a source 'marketing expert David Lee'""}",1,"Contains future tense words 'will likely increase' and specifies a date 'August 15, 2024', a target 'customer satisfaction ratings at Amazon', and a source 'marketing expert David Lee'",llama-3.1-8b-instant
1,"On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.","{""label"": 1, ""reasoning"": ""Contains the future tense words 'will likely increase' and a specific date 'August 15, 2024', indicating a prediction about the future""}",1,"Contains the future tense words 'will likely increase' and a specific date 'August 15, 2024', indicating a prediction about the future",llama-3.3-70b-versatile
2,"On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.","{""label"": 1, ""reasoning"": ""Contains the future tense words 'will likely increase' and a specific future date 'August 15, 2024', indicating a prediction about the future.""}",1,"Contains the future tense words 'will likely increase' and a specific future date 'August 15, 2024', indicating a prediction about the future.",llama-3.3-70b-instruct
3,"Operating profit for the three-month period increased from EUR1 .2 m , while revenue increased from EUR20 .2 m , as compared to the corresponding period in 2005 .","{""label"": 0, ""reasoning"": ""Does not contain a future tense word, it is a statement about a past comparison""}",0,"Does not contain a future tense word, it is a statement about a past comparison",llama-3.1-8b-instant
4,"Operating profit for the three-month period increased from EUR1 .2 m , while revenue increased from EUR20 .2 m , as compared to the corresponding period in 2005 .","{""label"": 0, ""reasoning"": ""The sentence is in the past tense, discussing increases in operating profit and revenue compared to a period in 2005, and does not contain any future tense words or phrases indicating a prediction about the future.""}",0,"The sentence is in the past tense, discussing increases in operating profit and revenue compared to a period in 2005, and does not contain any future tense words or phrases indicating a prediction about the future.",llama-3.3-70b-versatile
5,"Operating profit for the three-month period increased from EUR1 .2 m , while revenue increased from EUR20 .2 m , as compared to the corresponding period in 2005 .","{""label"": 0, ""reasoning"": ""The sentence is in the past tense, discussing increases in operating profit and revenue compared to a period in 2005, and does not contain any future tense words or phrases indicating a prediction about what will happen.""}",0,"The sentence is in the past tense, discussing increases in operating profit and revenue compared to a period in 2005, and does not contain any future tense words or phrases indicating a prediction about what will happen.",llama-3.3-70b-instruct
6,"Cybersecurity threats should stay same in 2026-06-01, according to the National Security Agency director, Dr. Sophia Rodriguez.","{""label"": 1, ""reasoning"": ""Contains the future tense phrase 'should stay same in 2026-06-01' indicating an expectation for the future, and a named source 'the National Security Agency director, Dr. Sophia Rodriguez'.""}",1,"Contains the future tense phrase 'should stay same in 2026-06-01' indicating an expectation for the future, and a named source 'the National Security Agency director, Dr. Sophia Rodriguez'.",llama-3.1-8b-instant
7,"Cybersecurity threats should stay same in 2026-06-01, according to the National Security Agency director, Dr. Sophia Rodriguez.","{

In [19]:
y_test_df.rename(columns={'Sentence Label' : 'Actual Label'}, inplace=True)

In [20]:
def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']


test_and_models_df = pd.concat([X_test_df.loc[:, :], y_test_df], axis=1)
# test_and_models_df.columns = ['Sentence', 'Actual Label']
for model in models:
    print(model.__name__())
    model_labels = get_llm_labels(results_with_llm_label_df, model.__name__())
    test_and_models_df[model.__name__()] = model_labels.to_numpy().ravel()
test_and_models_df

llama-3.1-8b-instant
llama-3.3-70b-versatile
llama-3.3-70b-instruct


,Base Sentence,Sentence Label,Embedding,Normalized Embeddings,Actual Label,llama-3.1-8b-instant,llama-3.3-70b-versatile,llama-3.3-70b-instruct
0,"On August 15, 2024, marketing expert David Lee speculates that the customer satisfaction ratings at Amazon will likely increase.",1,[-2.67128736e-01 3.99917126e-01 1.51789542e-02 -8.48717690e-02\n -9.08976942e-02 -2.59267748e-03 5.57870697e-03 -1.11984558e-01\n 5.50340861e-02 1.75547993e+00 -2.54974365e-01 -1.80750433e-02\n 2.53512263e-02 -5.75582273e-02 -1.17895454e-01 -3.12662870e-02\n -7.41217807e-02 1.10751092e+00 -1.34572908e-01 -2.01131892e-03\n 5.15997782e-02 1.16336204e-01 -8.87980685e-03 -1.33108839e-01\n 4.78714556e-02 5.70764579e-02 -1.72738910e-01 4.16884683e-02\n 2.07518153e-02 4.11881283e-02 7.36815408e-02 -1.50440494e-02\n 3.74726136e-03 1.82219267e-01 -6.20155334e-02 -4.09787893e-03\n 5.32624172e-03 8.10612142e-02 -4.86277193e-02 -1.43699899e-01\n -7.53155202e-02 -2.91622225e-02 8.86417702e-02 -5.73523762e-03\n 5.72941788e-02 5.58102839e-02 -1.44644111e-01 -6.10220470e-02\n 6....,[-1.92454374e+00 1.91188228e+00 6.52823076e-02 -1.09791911e+00\n -1.71294856e+00 5.92892647e-01 1.92068860e-01 -1.46676123e+00\n 1.23968937e-01 -3.15836072e-02 6.55651987e-01 -5.98727763e-01\n -3.60853940e-01 -9.42990124e-01 -7.83263743e-01 9.00564417e-02\n -1.02269506e+00 3.85037921e-02 -3.46616179e-01 3.87996107e-01\n 4.86750692e-01 8.39973629e-01 -1.23200588e-01 -6.20292664e-01\n 1.05327025e-01 -1.63882062e-01 -7.20112205e-01 2.79392928e-01\n -3.39587964e-02 -5.77544093e-01 1.50556660e+00 -4.31914985e-01\n 1.02688305e-01 1.54282737e+00 -2.37315607e+00 2.74423778e-01\n 4.23447758e-01 4.65204388e-01 -6.98811769e-01 -1.86601353e+00\n -1.40664899e+00 -1.21409535e+00 4.51989844e-02 3.71334434e-01\n 7.22978592e-01 9.42269504e-01 -1.18480968e+00 -2.19736397e-01\n 5....,1,1,1,1
1,"Operating profit for the three-month period increased from EUR1 .2 m , while revenue increased from EUR20 .2 m , as compared to the corresponding period in 2005 .",0,[-8.39660242e-02 1.80937588e-01 -4.57216762e-02 1.31758088e-02\n -1.15853874e-02 -5.86525165e-02 -8.70665710e-04 3.26534510e-02\n 1.16215296e-01 1.84254622e+00 -2.33168826e-01 1.67165652e-01\n -6.99797019e-05 -2.04802342e-02 4.41387780e-02 -8.78530294e-02\n 3.87666002e-02 1.38155854e+00 -1.93677381e-01 2.67565437e-02\n 8.30405392e-03 2.85588354e-02 -9.71042439e-02 -2.32131258e-02\n 4.73141745e-02 -5.10274470e-02 -9.36960801e-02 4.17356864e-02\n -1.44721614e-02 1.21533826e-01 -6.50100559e-02 -2.13543437e-02\n -8.56587365e-02 2.31346134e-02 4.07231599e-02 -7.77563825e-02\n -1.08038373e-01 -2.93932352e-02 2.08473653e-02 7.65744643e-03\n -1.03914235e-02 4.45170105e-02 7.58761391e-02 -4.45002131e-03\n -9.02475566e-02 1.67677191e-03 -5.21407127e-02 7.16749728e-02\n -2....,[ 0.32650337 -1.0380536 -0.91864634 0.51113987 -0.65212625 -0.29796603\n 0.07093044 0.41100782 0.9220639 0.2662444 0.95830584 1.7495745\n -0.7324779 -0.3097109 0.88611746 -0.9876781 1.0547875 1.9621172\n -1.3324827 0.875457 -0.40856755 -0.4626243 -1.5807649 0.94958717\n 0.0967818 -1.6057067 0.50081563 0.2801942 -0.6300226 0.58435524\n -1.0352651 -0.5324218 -1.302615 -0.7046011 -0.29519314 -0.9691\n -1.0828011 -1.2725768 0.4982816 0.4064646 -0.23578587 -0.05474434\n -0.12900919 0.39317027 -2.0725603 0.0272575 0.11121429 1.404259\n -0.9829128 0.3590206 0.32491672 -1.1381576 -0.5637724 0.83665544\n 0.79479074 -0.45303088 -0.28690377 -1.7320254 -1.0912753 1.3369948\n 0.07836758 -0.26163587 -0.870195 0.77505666 -1.885778 -0.8...,0,0,0,0
2,"Cybersecurity threats should stay same in 2026-06-01, according to the National Security Agency director, Dr. Sophia Rodriguez.",1,[-9.97022167e-02 2.27282479e-01 8.88563171e-02 1.93849038e-02\n 8.27627033e-02 -6.11012615e-03 6.32572249e-02 -1.25566736e-01\n -9.99972895e-02 1.72420835e+00 -3.31175238e-01 -4.78917100e-02\n 4.07287441e-02 -8.05337653e-02 -1.63093254e-01 -3.94880436e-02\n -1.27118714e-02 8.93040419e-01 -5.53227402e-02 7.60172233e-02\n 1.45164803e-01 1.

## Save Output

In [21]:
# save_path = os.path.join(combine_data_path )
DataProcessing.save_to_file(test_and_models_df, combine_data_path, 'llm_classifiers', '.csv')

Saved to: 
	/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_classification_experiments-v2/../data/combined_generated_fin_phrase_bank/llm_classifiers-v1.csv


## Evaluation

In [22]:
print("======= EVALUATION/RESULTS =======")

======= EVALUATION/RESULTS =======


In [23]:
get_metrics = EvaluationMetric()
get_metrics

In [24]:
actual_label = test_and_models_df['Actual Label'].values
for ml_model in models:
    ml_model_name = ml_model.__name__()
    print(f"Actual Label:\t\t{actual_label}")
    ml_model_predictions = test_and_models_df[ml_model_name].values
    print(f"{ml_model_name}:\t\t{ml_model_predictions}")
    get_metrics.eval_classification_report(y_test_df, ml_model_predictions)

Actual Label:		[1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0]
llama-3.1-8b-instant:		[1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0]
              precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       1.00      0.89      0.94        18

    accuracy                           0.90        21
   macro avg       0.80      0.94      0.85        21
weighted avg       0.94      0.90      0.91        21

Actual Label:		[1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0]
llama-3.3-70b-versatile:		[1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0]
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.94      0.97        18

    accuracy                           0.95        21
   macro avg       0.88      0.97      0.91        21
weighted avg       0.96      0.95      0.96        21

Actual Label:		[1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0]
llama-3.3-70b-instruct:		[1 0 1 